In [ ]:
import numpy as np
import os, time, tqdm, sys

# add parent dictionary to path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(parent_dir)
# create pybullet environment
from manipulation_env.env import ManiEnv
from manipulation_env.robot import UR10Robotiq85

robot_model_path = "../../manipulation_env/models/urdf/ur10_robotiq_85.urdf"
table_path = "../../manipulation_env/models/urdf/objects/table/table.urdf"
block_path = "../../manipulation_env/models/YcbPottedMeatCan/model.urdf"
robot = UR10Robotiq85(model_path=robot_model_path)
env = ManiEnv(robot, block_path=block_path, table_path=table_path, vis=True)
env.reset()

In [ ]:
# load the trajectory
# load the data 
pushing_results = np.load("reaching_trajecotries.npz", allow_pickle=True)
# calculate the success rate
success_rate = 0
for traj in pushing_results["tractories"]:
    if traj["success"]:
        success_rate += 1

print("Success rate: ", success_rate/len(pushing_results["tractories"]))

env.clean_traj_plot()
# visualize the trajectory
num_traj = 0
for traj in pushing_results["tractories"]:
    num_traj += 1
    env.reset_block(traj["block_pose"], euler_angle=[0, 0, 0])
    traj_data = traj["traj"]
    env.step(traj_data[:, 1])
    traj_data[2, :] = traj_data[2, :]
    env.traj_plot(traj_data[:3,:].T, color=[1, 0, 0])

    if num_traj == 1:
        # exute the first trajectory
        for i in range(traj_data.shape[1]):
            env.step(traj_data[:, i])        
        break

In [ ]:
# load pushing trajectories
pushing_results = np.load("pushing_trajecotries.npz", allow_pickle=True)
# calculate the success rate
success_rate = 0
for traj in pushing_results["tractories"]:
    if traj["success"]:
        success_rate += 1

print("Success rate: ", success_rate/len(pushing_results["tractories"]))

env.clean_traj_plot()
# visualize the trajectory
num_traj = 0
for traj in pushing_results["tractories"]:
    num_traj += 1
    env.reset_block(traj["block_pose"], euler_angle=[0, 0, np.pi/2])
    traj_data = traj["traj"]
    env.step(traj_data[:, 1])
    traj_data[2, :] = traj_data[2, :]
    env.traj_plot(traj_data[:3,:].T, color=[1, 0, 0])

    if num_traj == 1:
        # exute the first trajectory
        for i in range(traj_data.shape[1]):
            env.step(traj_data[:, i])
        break
        

In [ ]:
# find where failed, and store the failed trajectories
env.clean_traj_plot()
failed_pushing = []
block_pos = []
for traj in pushing_results["tractories"]:
    if not traj["success"]:
        failed_pushing.append(traj)
        block_pos.append(traj["block_pose"])

print("Number of failed pushing: ", len(failed_pushing))

# replay the failed pushing trajectories
# execute the pushing trajectory
for i in range(len(failed_pushing)):    
    env.reset_block(block_pos[i], euler_angle=[0, 0, np.pi/2])
    traj_data = failed_pushing[i]
    env.step(traj_data[:, 1])
    for j in range(traj_data.shape[1]):
        env.step(traj_data[:, j])



In [ ]:
# load grasping trajectories
grasping_results = np.load("grasping_trajecotries.npz", allow_pickle=True)
# calculate the success rate
success_rate = 0
for traj in grasping_results["tractories"]:
    if traj["success"]:
        success_rate += 1

print("Success rate: ", success_rate/len(grasping_results["tractories"]))
env.clean_traj_plot()
# visualize the trajectory
num_traj = 0
for traj in grasping_results["tractories"]:
    num_traj += 1
    env.reset_block(traj["block_pose"], euler_angle=[0, 0, 0])
    traj_data = traj["traj"]
    env.step(traj_data[:, 1])
    env.traj_plot(traj_data[:3,:].T, color=[1, 0, 0])

    if num_traj == 1:
        # exute the first trajectory
        for i in range(traj_data.shape[1]):
            env.step(traj_data[:, i])
        break

In [ ]:
env.clean_traj_plot()
failed_grasping = []
block_pos = []
block_orien = []
for traj in grasping_results["tractories"]:
    if not traj["success"]:
        failed_grasping.append(traj["traj"])
        block_pos.append(traj["block_pose"])
        # find keys other than "block_pose", "traj", "success"        
        for key in traj.keys():
            if key not in ["block_pose", "traj", "success"]:
                block_orien.append(key)

print("Number of failed grasping: ", len(failed_grasping))

# replay the failed pushing trajectories
# execute the pushing trajectory
for i in range(len(failed_grasping)):   
    env.clean_traj_plot() 
    env.reset_block(block_pos[i], euler_angle=[0, 0, block_orien[i]])
    traj_data = failed_grasping[i]
    env.step(traj_data[:, 1])
    env.traj_plot(traj_data[:3,:].T)
    for j in range(traj_data.shape[1]):
        env.step(traj_data[:, j])

        # check if the block is grasped
        if env.check_grasping():
            print("Grasping success")
            break

In [ ]:
# plot rotation and gripper value of all trajectory
env.clean_traj_plot()
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2, 1)
for traj in grasping_results["tractories"]:
    traj_data = traj["trajecoty"]
    ax[0].plot(traj_data[4, :])
    ax[1].plot(traj_data[3, :])    

# set the title
ax[0].set_title("Gripper value of all grasping trajectory")
ax[1].set_title("Rotation value of all grasping trajectory")

# set the x label, y label
ax[0].set_xlabel("Step")
ax[1].set_xlabel("Step")
ax[0].set_ylabel("Gripper value")
ax[1].set_ylabel("Rotation value")

# set the disctance between subplots
plt.subplots_adjust(hspace=0.5)